In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('used-car-price').config('spark.executor.memory', '6g').getOrCreate()

24/09/09 14:10:47 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/09 14:10:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 14:10:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_dir="data"
spark = SparkSession.builder.master('local[*]').appName('used-car-price')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

24/09/09 14:10:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.port', '46555'),
 ('spark.driver.host', 'langchain.ht.home'),
 ('spark.app.submitTime',

In [5]:
csv_file_path = '../data/train.csv'
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)


In [6]:
df.show()

+---+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------+--------------------+-----------+-----+
| id|        brand|               model|model_year|milage|    fuel_type|              engine|        transmission|             ext_col|int_col|            accident|clean_title|price|
+---+-------------+--------------------+----------+------+-------------+--------------------+--------------------+--------------------+-------+--------------------+-----------+-----+
|  0|         MINI|       Cooper S Base|      2007|213000|     Gasoline|172.0HP 1.6L 4 Cy...|                 A/T|              Yellow|   Gray|       None reported|        Yes| 4200|
|  1|      Lincoln|               LS V8|      2002|143250|     Gasoline|252.0HP 3.9L 8 Cy...|                 A/T|              Silver|  Beige|At least 1 accide...|        Yes| 4999|
|  2|    Chevrolet|   Silverado 2500 LT|      2002|136731|E85 Flex Fuel|320.0HP 5.3L 

In [7]:
df.select('model').show()

+--------------------+
|               model|
+--------------------+
|       Cooper S Base|
|               LS V8|
|   Silverado 2500 LT|
|    G90 5.0 Ultimate|
|         Metris Base|
|       A6 2.0T Sport|
|           A8 L 3.0T|
|  Silverado 1500 1LZ|
|           F-150 XLT|
|             M4 Base|
|          Camaro 1LT|
|       Model S P100D|
|Escalade ESV Plat...|
|S4 3.0T Premium Plus|
|Rover Range Rover...|
|          AMG C 63 S|
|        Yukon Denali|
|   Rover Defender SE|
|          Tundra SR5|
|       AMG C 63 Base|
+--------------------+
only showing top 20 rows



In [8]:
df_copy =  df

In [9]:
df_copy.columns

['id',
 'brand',
 'model',
 'model_year',
 'milage',
 'fuel_type',
 'engine',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'price']

replacing the value model feature with rare feature if their count is 1

In [10]:
from pyspark.sql.functions import when, col, count
model_counts = df_copy.groupBy('model').agg(count('model').alias('count'))
df_copy = df_copy.join(model_counts, on='model', how='left')
df_copy = df_copy.withColumn('model', when(col('count') == 1, 'Rare').otherwise(col('model')))
df_copy = df_copy.drop('count')

In [11]:
model_counts = df_copy.groupBy('model').agg(count('model').alias('count')).orderBy('count')
model_counts.filter(col('count') > 3).show(n=52)

+--------------------+-----+
|               model|count|
+--------------------+-----+
|             i3 Base|    4|
|            3500 SLT|    4|
|        X6 sDrive35i|    4|
|          Yaris Base|    4|
|        Type 57 Base|    4|
|C-Class 4MATIC Sedan|    4|
|     Ram 1500 SRT-10|    4|
|     Impreza Premium|    4|
|               Rio S|    4|
|                Rare|    4|
|allroad 2.0T Pres...|    4|
|Convertible John ...|    4|
|           e-Golf SE|    4|
|Niro Plug-In Hybr...|    5|
|       AMG C 43 Base|    5|
|             X7 M50i|    5|
|           Capri XR2|    5|
|     Maverick Lariat|    5|
|Model 3 Standard ...|    5|
|    XC70 T6 Platinum|    5|
|        bZ4X Limited|    5|
| HUMMER EV Edition 1|    5|
|Wagoneer Series I...|    5|
|   Corolla Hybrid LE|    5|
|           Mirage ES|    5|
|           850 Turbo|    5|
|   124 Spider Abarth|    5|
|   Countryman Cooper|    5|
|             S-10 LS|    5|
|Sorento Plug-In H...|    6|
|        X3 xDrive28i|    6|
|      e-tron 

replace the rear features in engine by rare columns

In [12]:
engine_counts = df_copy.groupBy('engine').agg(count('model').alias('count'))
df_copy = df_copy.join(engine_counts, on='engine', how='left')
df_copy = df_copy.withColumn('engine', when(col('count') == 1, 'Rare').otherwise(col('engine')))
df_copy = df_copy.drop('count')

In [13]:
engine_counts = df_copy.groupBy('engine').agg(count('engine').alias('count')).orderBy('count')
engine_counts.filter(col('count') > 7).show(n=52)

+--------------------+-----+
|              engine|count|
+--------------------+-----+
|    Battery Electric|    8|
|303.0HP 3.4L Flat...|    8|
|145.0HP 1.8L 4 Cy...|    8|
|2.0L I4 16V GDI D...|    8|
|192.0HP 1.5L 4 Cy...|    8|
|137.0HP 1.8L 4 Cy...|    8|
|3.2L V6 18V MPFI ...|    8|
|250.0HP 3.2L V6 C...|    8|
|1.3L I3 12V MPFI ...|    8|
|500.0HP 5.4L 8 Cy...|    8|
|  12 Cylinder Engine|    8|
|335.0HP 6.6L 8 Cy...|    8|
|214.0HP Electric ...|    8|
|                Rare|    8|
|210.0HP 2.0L 4 Cy...|    8|
|444.0HP 4.7L 8 Cy...|    8|
|495.0HP 5.0L 8 Cy...|    8|
|111.2Ah / FR 70kW...|    8|
|245.0HP 2.5L 4 Cy...|    8|
|76.0HP 1.5L 4 Cyl...|    8|
|193.0HP 2.5L 4 Cy...|    8|
|638.0HP 6.2L 8 Cy...|    8|
|320.0HP 4.6L 8 Cy...|    8|
|317.0HP 5.6L 8 Cy...|    9|
|451.0HP 6.3L 8 Cy...|    9|
|321.0HP Electric ...|    9|
|430.0HP 4.7L 8 Cy...|    9|
|175.0HP 1.6L 4 Cy...|    9|
|161.0HP 2.0L 4 Cy...|    9|
|308.0HP 3.6L V6 C...|    9|
|180.0HP 3.4L V6 C...|    9|
|182.0HP Elect

In [14]:
from pyspark.sql.functions import regexp_replace

# Replacing special characters 
df_copy = df_copy.withColumn(
    'engine', 
    regexp_replace('engine', '[^a-zA-Z0-9\s]', '') 
)


In [15]:
engine_counts = df_copy.groupBy('engine').agg(count('engine').alias('count')).orderBy('count')
engine_counts.filter(col('count') > 7).show(n=52)

+--------------------+-----+
|              engine|count|
+--------------------+-----+
|4440HP 47L 8 Cyli...|    8|
|13L I3 12V MPFI D...|    8|
|1370HP 18L 4 Cyli...|    8|
|    Battery Electric|    8|
|3030HP 34L Flat 6...|    8|
|20L I4 16V GDI DO...|    8|
|1920HP 15L 4 Cyli...|    8|
|32L V6 18V MPFI SOHC|    8|
|2500HP 32L V6 Cyl...|    8|
|3350HP 66L 8 Cyli...|    8|
|  12 Cylinder Engine|    8|
|1450HP 18L 4 Cyli...|    8|
|2100HP 20L 4 Cyli...|    8|
|2140HP Electric M...|    8|
|5000HP 54L 8 Cyli...|    8|
|                Rare|    8|
|1930HP 25L 4 Cyli...|    8|
|1112Ah  FR 70kW  ...|    8|
|2450HP 25L 4 Cyli...|    8|
|4950HP 50L 8 Cyli...|    8|
|760HP 15L 4 Cylin...|    8|
|6380HP 62L 8 Cyli...|    8|
|3200HP 46L 8 Cyli...|    8|
|1610HP 20L 4 Cyli...|    9|
|1800HP 34L V6 Cyl...|    9|
|3210HP Electric M...|    9|
|1750HP 16L 4 Cyli...|    9|
|3170HP 56L 8 Cyli...|    9|
|4300HP 47L 8 Cyli...|    9|
|4510HP 63L 8 Cyli...|    9|
|1820HP Electric M...|    9|
|3080HP 36L V6

engineering the transmisson column

In [16]:
df_copy

DataFrame[engine: string, model: string, id: int, brand: string, model_year: int, milage: int, fuel_type: string, transmission: string, ext_col: string, int_col: string, accident: string, clean_title: string, price: int]

In [17]:
df_copy = df_copy.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
df_copy.show(n=52)

+--------------------+-----+
|        transmission|count|
+--------------------+-----+
|SCHEDULED FOR OR ...|    6|
|         8-SPEED A/T|    7|
|          8-SPEED AT|   13|
|               CVT-F|   15|
|Single-Speed Fixe...|   16|
|7-Speed DCT Autom...|   18|
|       6 Speed At/Mt|   20|
|   2-Speed Automatic|   22|
|              Manual|   27|
|             6-Speed|   27|
|      8-Speed Manual|   28|
|             7-Speed|   29|
|6-Speed Electroni...|   31|
|9-Speed Automatic...|   33|
|          6 Speed Mt|   37|
|10-Speed Automati...|   40|
|   4-Speed Automatic|   48|
|                   F|   65|
|            Variable|   66|
|         2-Speed A/T|   71|
|                   –|   87|
|                   2|   91|
|6-Speed Automatic...|  111|
|      7-Speed Manual|  151|
|Transmission Over...|  156|
|   5-Speed Automatic|  159|
|   7-Speed Automatic|  178|
|      6-Speed Manual|  211|
|         7-Speed M/T|  355|
|   1-Speed Automatic|  365|
|8-Speed Automatic...|  549|
|       Automa

In [24]:
from pyspark.sql import functions as F
# 1. Lowercase all values
df_clean = df.withColumn(
    "transmission_clean",
    F.lower(F.col("transmission"))
)

# 2. Remove special characters and unwanted text
df_clean = df_clean.withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"[^a-z0-9\s-]", "")
)

# 3. Trim any leading or trailing spaces
df_clean = df_clean.withColumn(
    "transmission_clean",
    F.trim(F.col("transmission_clean"))
)

# 4. Normalize transmission types by replacing common abbreviations and patterns
df_clean = df_clean.withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"a/t", "automatic")
).withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"m/t", "manual")
).withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"\bat\b", "automatic")
).withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"\bmt\b", "manual")
)

# 5. Normalize spacing (handle cases like "6 Speed Mt" -> "6-Speed Manual")
df_clean = df_clean.withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"\s+", " ")  
).withColumn(
    "transmission_clean",
    F.regexp_replace(F.col("transmission_clean"), r"(\d+)\s*speed", "$1-speed")  
)

# 6. Group similar transmission types for consistent values
df_clean = df_clean.withColumn(
    "transmission_clean",
    F.when(F.col("transmission_clean").contains("1-speed") & F.col("transmission_clean").contains("manual"), "1-speed manual")
    .when(F.col("transmission_clean").contains("1-speed") & F.col("transmission_clean").contains("automatic"), "1-speed automatic")
    
    .when(F.col("transmission_clean").contains("2-speed") & F.col("transmission_clean").contains("manual"), "2-speed manual")
    .when(F.col("transmission_clean").contains("2-speed") & F.col("transmission_clean").contains("automatic"), "2-speed automatic")
    
    .when(F.col("transmission_clean").contains("4-speed") & F.col("transmission_clean").contains("manual"), "4-speed manual")
    .when(F.col("transmission_clean").contains("4-speed") & F.col("transmission_clean").contains("automatic"), "4-speed automatic")
    
    .when(F.col("transmission_clean").contains("5-speed") & F.col("transmission_clean").contains("manual"), "5-speed manual")
    .when(F.col("transmission_clean").contains("5-speed") & F.col("transmission_clean").contains("automatic"), "5-speed automatic")
    
    .when(F.col("transmission_clean").contains("6-speed") & F.col("transmission_clean").contains("at/mt"), "6-speed automatic/manual")
    .when(F.col("transmission_clean").contains("6-speed") & F.col("transmission_clean").contains("manual"), "6-speed manual")
    .when(F.col("transmission_clean").contains("6-speed") & F.col("transmission_clean").contains("automatic"), "6-speed automatic")
    .when(F.col("transmission_clean").contains("6-speed") & F.col("transmission_clean").contains("electronic"), "6-speed automatic")
    
    .when(F.col("transmission_clean").contains("7-speed") & F.col("transmission_clean").contains("manual"), "7-speed manual")
    .when(F.col("transmission_clean").contains("7-speed") & F.col("transmission_clean").contains("automatic"), "7-speed automatic")
    
    .when(F.col("transmission_clean").contains("8-speed") & F.col("transmission_clean").contains("manual"), "8-speed manual")
    .when(F.col("transmission_clean").contains("8-speed") & F.col("transmission_clean").contains("automatic"), "8-speed automatic")
    
    .when(F.col("transmission_clean").contains("9-speed") & F.col("transmission_clean").contains("manual"), "9-speed manual")
    .when(F.col("transmission_clean").contains("9-speed") & F.col("transmission_clean").contains("automatic"), "9-speed automatic")
    
    .when(F.col("transmission_clean").contains("10-speed") & F.col("transmission_clean").contains("manual"), "10-speed manual")
    .when(F.col("transmission_clean").contains("10-speed") & F.col("transmission_clean").contains("automatic"), "10-speed automatic")

    # Handling mixed cases like "6 Speed At/Mt" or "6 Speed Mt"
    .when(F.col("transmission_clean").contains("at/mt"), "automatic/manual")
    
    # Default case for other transmissions
    .otherwise(F.col("transmission_clean"))
)

df_clean.groupBy("transmission_clean").agg(F.count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+-----------------------------+-----+
|transmission_clean           |count|
+-----------------------------+-----+
|automatic                    |60595|
|8-speed automatic            |29645|
|6-speed automatic            |20985|
|transmission wdual shift mode|19255|
|7-speed automatic            |12492|
|6-speed manual               |12246|
|10-speed automatic           |11248|
|9-speed automatic            |6224 |
|5-speed automatic            |3376 |
|4-speed automatic            |2594 |
|5-speed manual               |2409 |
|1-speed automatic            |2234 |
|cvt transmission             |1990 |
|manual                       |1206 |
|automatic cvt                |829  |
|7-speed manual               |506  |
|transmission overdrive switch|156  |
|2-speed automatic            |93   |
|2                            |91   |
|                             |87   |
+-----------------------------+-----+
only showing top 20 rows



In [19]:
df_new = df_clean.groupBy('transmission_clean').agg(count('transmission_clean').alias('count')).orderBy('count')
df_new.show(n=52)

+--------------------+-----+
|  transmission_clean|count|
+--------------------+-----+
|scheduled for or ...|    6|
|               cvt-f|   15|
|single-speed fixe...|   16|
|        6-speed atmt|   20|
|             6-speed|   27|
|      8-speed manual|   28|
|             7-speed|   29|
|                   f|   65|
|            variable|   66|
|                    |   87|
|                   2|   91|
|   2-speed automatic|   93|
|transmission over...|  156|
|      7-speed manual|  506|
|       automatic cvt|  829|
|              manual| 1206|
|    cvt transmission| 1990|
|   1-speed automatic| 2234|
|      5-speed manual| 2409|
|   4-speed automatic| 2594|
|   5-speed automatic| 3376|
|   9-speed automatic| 6224|
|  10-speed automatic|11248|
|      6-speed manual|12246|
|   7-speed automatic|12492|
|transmission wdua...|19255|
|   6-speed automatic|20985|
|   8-speed automatic|29645|
|           automatic|60595|
+--------------------+-----+



In [27]:
df_new

DataFrame[transmission_clean: string, count: bigint]

In [ ]:
out

In [28]:
df_merged = df_clean.join(df_new, df["transmission"] == df_new["transmission_clean"], "inner")

# Drop the original 'transmission' column
df_merged = df_merged.drop(df["transmission"])

# Rename 'transmission_clean' to 'transmission' for consistency
df_merged = df_merged.withColumnRenamed("transmission_clean", "transmission")

# Show the resulting merged dataframe
df_merged.show()

24/09/09 14:33:48 ERROR CodeGenerator: failed to compile: org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 146, Column 14: Compiling "hashAgg_doAggregateWithKeys_0()"
org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 146, Column 14: Compiling "hashAgg_doAggregateWithKeys_0()"
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:402)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:236)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:363)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:361)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:371)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:361)
	at org.codehaus.janino.SimpleCompiler.cook(SimpleCompiler.java:2

+-----+-----+--------------------+----------+------+---------+--------------------+--------------------+-------------------+--------------------+-----------+-----+------------+------------+-----+
|   id|brand|               model|model_year|milage|fuel_type|              engine|             ext_col|            int_col|            accident|clean_title|price|transmission|transmission|count|
+-----+-----+--------------------+----------+------+---------+--------------------+--------------------+-------------------+--------------------+-----------+-----+------------+------------+-----+
| 2187|Acura|RDX w/Advance Pac...|      2021| 33705| Gasoline|           2.0 Liter|                   –|                  –|       None reported|       NULL|42598|           2|           2|   91|
| 3038|Acura|RDX w/A-Spec Package|      2021| 13613| Gasoline|     2.0 Liter Turbo|Majestic Black Pearl|              Black|       None reported|       NULL|27798|           2|           2|   91|
| 5642|Acura|     RD

In [ ]:

df_new = df_copy.withColumn(
    'transmission', 
    regexp_replace('transmission', '-', ' ') 
)

In [ ]:
df_copy.select('transmission').distinct().show(n=100)


+--------------------+
|        transmission|
+--------------------+
|SCHEDULED FOR OR ...|
|    CVT Transmission|
|8-Speed Automatic...|
|Transmission Over...|
|                   F|
|          6 Speed Mt|
|7-Speed Automatic...|
|      7-Speed Manual|
|6-Speed Electroni...|
|               CVT-F|
|         2-Speed A/T|
|   6-Speed Automatic|
|9-Speed Automatic...|
|           Automatic|
|      8-Speed Manual|
|                 A/T|
|   1-Speed Automatic|
|   9-Speed Automatic|
|                   –|
|         4-Speed A/T|
|   5-Speed Automatic|
|Single-Speed Fixe...|
|        10-Speed A/T|
|         9-Speed A/T|
|  10-Speed Automatic|
|         7-Speed M/T|
|   7-Speed Automatic|
|         5-Speed A/T|
|   8-Speed Automatic|
|10-Speed Automati...|
|         5-Speed M/T|
|                 M/T|
|      6-Speed Manual|
|7-Speed DCT Autom...|
|              Manual|
|         8-Speed A/T|
|             6-Speed|
|6-Speed Automatic...|
|       6 Speed At/Mt|
|         8-SPEED A/T|
|         1

In [ ]:
df_new = df_new.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
df_new.show(n=52)

+--------------------+-----+
|        transmission|count|
+--------------------+-----+
|SCHEDULED FOR OR ...|    1|
|8 Speed Automatic...|    1|
|    CVT Transmission|    1|
|             7 Speed|    1|
|             6 Speed|    1|
|Transmission Over...|    1|
|                   F|    1|
|          6 Speed Mt|    1|
|   5 Speed Automatic|    1|
|         8 SPEED A/T|    1|
|   4 Speed Automatic|    1|
|   1 Speed Automatic|    1|
|         6 Speed M/T|    1|
|   6 Speed Automatic|    1|
|   9 Speed Automatic|    1|
|           Automatic|    1|
|                 A/T|    1|
|               CVT F|    1|
|                   –|    1|
|6 Speed Automatic...|    1|
|   2 Speed Automatic|    1|
|7 Speed DCT Autom...|    1|
|         2 Speed A/T|    1|
|      6 Speed Manual|    1|
|   8 Speed Automatic|    1|
|                 M/T|    1|
|         6 Speed A/T|    1|
|         5 Speed M/T|    1|
|         7 Speed M/T|    1|
|7 Speed Automatic...|    1|
|      8 Speed Manual|    1|
|             

+--------------------+-----+
|        transmission|count|
+--------------------+-----+
|SCHEDULED FOR OR ...|    6|
|         8-SPEED A/T|    7|
|          8-SPEED AT|   13|
|               CVT-F|   15|
|Single-Speed Fixe...|   16|
|7-Speed DCT Autom...|   18|
|       6 Speed At/Mt|   20|
|   2-Speed Automatic|   22|
|              Manual|   27|
|             6-Speed|   27|
|      8-Speed Manual|   28|
|             7-Speed|   29|
|6-Speed Electroni...|   31|
|9-Speed Automatic...|   33|
|          6 Speed Mt|   37|
|10-Speed Automati...|   40|
|   4-Speed Automatic|   48|
|                   F|   65|
|            Variable|   66|
|         2-Speed A/T|   71|
|                   –|   87|
|                   2|   91|
|6-Speed Automatic...|  111|
|      7-Speed Manual|  151|
|Transmission Over...|  156|
|   5-Speed Automatic|  159|
|   7-Speed Automatic|  178|
|      6-Speed Manual|  211|
|         7-Speed M/T|  355|
|   1-Speed Automatic|  365|
|8-Speed Automatic...|  549|
|       Automa

In [ ]:
df_new = df_copy.withColumn(
    'transmission',
    regexp_replace(df_copy['transmission'], '-', ' ')
)

In [ ]:
df_new = df_new.withColumn(
    'transmission',
    regexp_replace(col('transmission'), '  ', ' ')  # Removing extra spaces
)


In [ ]:
df_new = df_new.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
df_new.show(n=52)

+--------------------+-----+
|        transmission|count|
+--------------------+-----+
|SCHEDULED FOR OR ...|    1|
|8 Speed Automatic...|    1|
|    CVT Transmission|    1|
|             7 Speed|    1|
|             6 Speed|    1|
|Transmission Over...|    1|
|                   F|    1|
|          6 Speed Mt|    1|
|   5 Speed Automatic|    1|
|         8 SPEED A/T|    1|
|   4 Speed Automatic|    1|
|   1 Speed Automatic|    1|
|         6 Speed M/T|    1|
|   6 Speed Automatic|    1|
|   9 Speed Automatic|    1|
|           Automatic|    1|
|                 A/T|    1|
|               CVT F|    1|
|                   –|    1|
|6 Speed Automatic...|    1|
|   2 Speed Automatic|    1|
|7 Speed DCT Autom...|    1|
|         2 Speed A/T|    1|
|      6 Speed Manual|    1|
|   8 Speed Automatic|    1|
|                 M/T|    1|
|         6 Speed A/T|    1|
|         5 Speed M/T|    1|
|         7 Speed M/T|    1|
|7 Speed Automatic...|    1|
|      8 Speed Manual|    1|
|             

In [ ]:

# df_copy = df_copy.withColumn(
#     'transmission',
#     regexp_replace(df_copy['transmission'], 'A/T', 'AT')
# )

In [ ]:
# df_copy = df_copy.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
# df_copy.show(n=52)

In [ ]:
# df_copy = df_copy.withColumn(
#     'transmission',
#     regexp_replace(df_copy['transmission'], 'Automatic', 'AT')
# )


In [ ]:
# df_copy = df_copy.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')

In [ ]:
# df_copy = df_copy.withColumn(
#     'transmission',
#     regexp_replace(df_copy['transmission'], 'Manual', 'Mt')
# )

# df_copy = df_copy.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
# df_copy.show(n=52)